In [1]:
import warnings
import os

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from datetime import datetime
from tqdm import tqdm

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

In [2]:
def parse_datetime(dt):
    time_formats = ["%d/%m/%Y %H:%M:%S %p", "%Y-%m-%d %H:%M:%S", '%Y-%m-%d %H:%M:%S.%f']

    for fmt in time_formats:
        try:
            return datetime.strptime(dt, fmt)
        except ValueError:
            pass
    
    raise ValueError('no valid date format found')

def preprocess(file_path: str ) -> pd.DataFrame:
    "preprocesses the raw dataset into a usable format for data science"
    # Read in dataframe
    for header_row in [0,1,2,3]:
        df_raw = pd.read_excel(file_path, header=header_row)
        if 'PriceUpdatedDate' in df_raw.columns:
            break
    
    # Filter df, convert to dt
    try:
        df = df_raw[df_raw['FuelCode']=="U91"]
    except KeyError:
        df = df_raw[df_raw['FuelType']=="U91"]

    df['long_date'] = df['PriceUpdatedDate'].apply(lambda x: parse_datetime(str(x)))
    df['price'] = df['Price']
    df = df[['long_date', 'price']].sort_values('long_date').reset_index(drop=True)

    # Create day, month, year vars
    df['day'] = df['long_date'].apply(lambda x: x.day)
    df['month'] = df['long_date'].apply(lambda x: x.month)
    df['year'] = df['long_date'].apply(lambda x: x.year)

    # Get mean obs for each day
    df_agg = df.groupby(['day', 'month', 'year']).mean().reset_index()
    df_agg['date'] = df_agg.apply(lambda row: datetime(year=int(row['year']), month=int(row['month']), day=int(row['day'])), axis=1)
    
    return df_agg[['date', 'price']]

def bind_dfs(raw_data):
    df_all = pd.concat(list(raw_data.values()))
    return pd.DataFrame(df_all.groupby('date')['price'].mean()).sort_values('date').reset_index()

In [ ]:
RAW_DATA = {}
for i, filename in tqdm(enumerate(os.listdir('data'))):
    if i <= 0:
        df_raw = preprocess(f'data/{filename}')
        RAW_DATA[filename] = df_raw
        print(f"\U00002705 Processed {filename}. nrows = {len(df_raw)}")

In [ ]:
df_march = pd.read_csv("march_data.csv")
df_march['date'] = pd.to_datetime(df_march['date'])
RAW_DATA['march2023'] = df_march

In [ ]:
df_all = bind_dfs(RAW_DATA)
df_all

In [ ]:
df_all.to_csv('data.csv')